In [126]:
print("RAG SCRAP")

RAG SCRAP


In [127]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()

True

In [128]:
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [129]:
llm=ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3
)

In [130]:
llm.invoke('What you know about cyber security?').content

"Cybersecurity! It's a vast and fascinating field that protects computer systems, networks, and sensitive data from unauthorized access, use, disclosure, disruption, modification, or destruction. Here's a comprehensive overview of what I know:\n\n**Key Concepts:**\n\n1. **Confidentiality**: Protecting sensitive information from unauthorized access.\n2. **Integrity**: Ensuring data accuracy and preventing modifications or deletions.\n3. **Availability**: Ensuring systems and data are accessible when needed.\n\n**Types of Cyber Threats:**\n\n1. **Malware**: Software designed to harm or exploit systems (e.g., viruses, worms, trojans, ransomware).\n2. **Phishing**: Social engineering attacks using fake emails, messages, or websites to trick users into revealing sensitive information.\n3. **DDoS (Distributed Denial of Service)**: Overwhelming a system with traffic to make it unavailable.\n4. **SQL Injection**: Injecting malicious code into databases to access or modify sensitive data.\n5. *

In [131]:
from bs4 import BeautifulSoup
import requests

def web_scrap(url:str)->str:
    response=requests.get(url)
    html_content=response.text
    soup = BeautifulSoup(html_content, "lxml")
    text = soup.get_text()
    text = ' '.join(text.split())
    if text:
        return text



In [132]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embed=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [133]:
from langchain.docstore.document import Document

In [134]:
web_text=web_scrap("https://www.daraz.pk/catalog/?spm=a2a0e.tm80335142.search.d_go&q=Hp%20laptop")

In [135]:
web_doc=[Document(page_content=web_text,metadata={'source':'web Scraping'})]
web_doc

[Document(metadata={'source': 'web Scraping'}, page_content='Buy Hp laptop Online at Best Price in Pakistan - Daraz.pk Save More on App Download the App Sell On Daraz Help & Support Help Center Contact Customer Care Order Shipping & Delivery Payment Returns & Refunds Login Sign Up Manage My Account My Orders My Wishlist & Followed Stores My Reviews My Returns & Cancellations Logout زبان تبدیل کریں UR / Urdu EN / English Categories Customer Care Help Center How to Buy Corporate & Bulk Purchasing Returns & Refunds Daraz Shop Contact Us Purchase Protection Daraz Pick up Points Daraz About Us Digital Payments Daraz Donates Daraz Blog Terms & Conditions Privacy Policy NTN Number : 4012118-6 STRN Number : 1700401211818 Online Shopping App Online Grocery Shopping Daraz Exclusive Daraz University Sell on Daraz Join Daraz Affiliate Program Happy Shopping Download App Payment Methods Verified by Daraz International Pakistan Bangladesh Sri Lanka Myanmar Nepal Follow Us © Daraz 2025')]

In [136]:
splitter=RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=30)

In [147]:
chunks=splitter.split_documents(web_doc)

In [148]:
for c in chunks:
    print(c)
    print('-'*10)

page_content='Buy Hp laptop Online at Best Price in Pakistan - Daraz.pk Save More on App Download the App Sell On Daraz Help & Support Help Center Contact Customer Care Order Shipping & Delivery Payment Returns & Refunds Login Sign Up Manage My Account My Orders My Wishlist & Followed Stores My Reviews My Returns' metadata={'source': 'web Scraping'}
----------
page_content='Stores My Reviews My Returns & Cancellations Logout زبان تبدیل کریں UR / Urdu EN / English Categories Customer Care Help Center How to Buy Corporate & Bulk Purchasing Returns & Refunds Daraz Shop Contact Us Purchase Protection Daraz Pick up Points Daraz About Us Digital Payments Daraz Donates Daraz' metadata={'source': 'web Scraping'}
----------
page_content='Payments Daraz Donates Daraz Blog Terms & Conditions Privacy Policy NTN Number : 4012118-6 STRN Number : 1700401211818 Online Shopping App Online Grocery Shopping Daraz Exclusive Daraz University Sell on Daraz Join Daraz Affiliate Program Happy Shopping Downloa

In [149]:
vectorstores=Chroma.from_documents(chunks,embed)
retriever=vectorstores.as_retriever(search_type='similarity',search_kwargs={'k':4})

In [150]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory()

In [151]:
template="""You are a web Scraper agent. You will get a context and reply based on it.
response should not be out of context and not more than 50 words.
Here is input: {question}
Here is context: {context}"""
prompt=PromptTemplate(template=template,input_variables=['question','context'])

In [152]:
rag_web_scrap=RetrievalQA.from_chain_type(llm=llm,memory=memory,retriever=retriever,chain_type_kwargs={'prompt':prompt})

In [153]:
query='What is this?'

In [154]:
response=rag_web_scrap.invoke(query)

In [155]:
print('-----------RAG WEB SCRAPER---------')
print(response['result'])

-----------RAG WEB SCRAPER---------
It appears to be a list of languages, likely from a Wikipedia or similar webpage, given the mention of Creative Commons and Terms of Use at the bottom.


In [156]:
similarity=vectorstores.similarity_search(query,k=4)
for s in similarity:
    print(s)
    print('-'*10)

page_content='Gaeilge GÃ idhlig Ú¯ÛÙÚ©Û àªà«àªàª°àª¾àª¤à« Hak-kÃ¢-ngÃ® / å®¢å®¶èª Hausa Hornjoserbsce Ido Igbo Ilokano Interlingua Interlingue ÐÑÐ¾Ð½ Ãslenska Jawa à²à²¨à³à²¨à²¡ Kapampangan áá¶áá¶ááááá Kotava KreyÃ²l Ayisyen KurdÃ® / ÙÙØ±Ø¯Û Ú©ÙØ±Ø¯ÛÛ ÙØ§ÙÛÙØ¯Û' metadata={'source': 'web Scraping'}
----------
page_content='This page is available under the Creative Commons Attribution-ShareAlike License Terms of Use Privacy Policy' metadata={'source': 'web Scraping'}
----------
page_content='à¥ Ù ØµØ±Ù Bahasa Melayu Bahaso Minangkabau áá¼ááºáá¬áá¬áá¬ Nederlands æ¥æ¬èª Norsk (bokmÃ¥l) Norsk (nynorsk) ÐÐ¾Ñ ÑÐ¸Ð¹Ð½ OÊ»zbekcha / ÐÐ·Ð±ÐµÐºÑÐ° Polski PortuguÃªs ÒÐ°Ð·Ð°ÒÑÐ° / QazaqÅa / ÙØ§Ø²Ø§ÙØ´Ø§ RomÃ¢nÄ Simple English Sinugboanong Binisaya SlovenÄina' metadata={'source': 'web Scraping'}
----------
page_content='NÄhuatlahtÅlli NaijÃ¡ Nedersaksisch Nouormand / Normaund Novial Afaan Oromoo áá¡á­á¯ááºááá¬áá